## Практическое задание

<ol>
    <li>Попробуйте обучить нейронную сеть на Keras с другими параметрами. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?</li>
    <li>Поработайте с документацией Keras.</li>
</ol>

In [45]:
import tensorflow as tf

In [58]:
import os
import random as rn

In [72]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
sd = 1 # Here sd means seed.
np.random.seed(sd)
rn.seed(sd)
os.environ['PYTHONHASHSEED']=str(sd)
tf.random.set_seed(sd)

In [73]:
import keras

In [91]:
from keras.layers.normalization import BatchNormalization

In [92]:
# The full neural network code!
###############################
import numpy as np
import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical


train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

# Build the model.
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001), input_shape=(784,)),
    keras.layers.normalization.BatchNormalization(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001)),
    keras.layers.normalization.BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001)),
    keras.layers.normalization.BatchNormalization(),
    Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00001)),
    keras.layers.normalization.BatchNormalization(),
    Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.00001)),
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.85)


# Compile the model.
model.compile(
    #optimizer='adam',
    optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

# Train the model.
model.fit(
    train_images,
    to_categorical(train_labels),
    epochs=10,
    batch_size=32,
    validation_data=(test_images, to_categorical(test_labels))
)

# Evaluate the model.
results = model.evaluate(
    test_images,
    to_categorical(test_labels),
    batch_size=32
)

print('Test loss: {:10.4f}\nTest accuracy {:7.4f}'.format(results[0], results[1]))

# Save the model to disk.
model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
#predictions = model.predict(test_images[:5])

# Print our model's predictions.
#print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
#print(test_labels[:5]) # [7, 2, 1, 0, 4]

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 204us/step - loss: 0.3022 - accuracy: 0.9130 - val_loss: 0.2010 - val_accuracy: 0.9405
Epoch 2/10
60000/60000 [==============================] - 11s 185us/step - loss: 0.1653 - accuracy: 0.9530 - val_loss: 0.1465 - val_accuracy: 0.9547
Epoch 3/10
60000/60000 [==============================] - 12s 196us/step - loss: 0.1376 - accuracy: 0.9603 - val_loss: 0.1090 - val_accuracy: 0.9669
Epoch 4/10
60000/60000 [==============================] - 12s 192us/step - loss: 0.1205 - accuracy: 0.9650 - val_loss: 0.0982 - val_accuracy: 0.9722
Epoch 5/10
60000/60000 [==============================] - 12s 202us/step - loss: 0.1093 - accuracy: 0.9686 - val_loss: 1.5641 - val_accuracy: 0.6753
Epoch 6/10
60000/60000 [==============================] - 12s 194us/step - loss: 0.1051 - accuracy: 0.9698 - val_loss: 0.1088 - val_accuracy: 0.9693
Epoch 7/10
60000/60000 [==============================] 

In [93]:
# Для повышения точности было реализовано: 
#1. Количество эпох увеличино до 10
#2. Добавлен ExponentialDecay для постепенного снижения learning_rate и подобраны его параметры
#3. Количество нейронов в 1 слое было увеличино с 64 до 256
#4. Добавлен второй слой с количеством нейронов: 128
#5. Добавлена l2 регуляризация
#6. Добавлен BatchNormalization
#7. Добавлен слой с количеством нейронов: 32

Результат: test loss = 0.0894; test accuracy = 0.9764

В качестве функции для разбора предлагаю tf.keras.preprocessing.image.ImageDataGenerator